In [1]:
%load_ext autoreload
%autoreload 2
import predict
import glob
import skimage
import sklearn
import numpy as np
from sklearn import pipeline

In [2]:
# y = []
# irawsymbols = []
# datadir = 'data/MYDATA/train/*.png'
# for name in glob.glob(datadir):
#     symbol = name.split('_')[1].replace(".png","")
#     ilabels, irawsymbol = predict.file_to_raw_symbols(name, single_symbol=True) 
#     irawsymbol = np.squeeze(irawsymbol)
#     irawsymbols.append(irawsymbol)
#     y.append(symbol)
    
# sklearn.externals.joblib.dump(irawsymbols, 'data/MYDATA/irawsymbols.p')
# sklearn.externals.joblib.dump(y, 'data/MYDATA/y.p')

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.externals import joblib
y = joblib.load('data/MYDATA/symbols.p')
X = joblib.load('data/MYDATA/irawsymbols.p')

In [4]:
def myresize(images, image_size):
    inp_processedz = np.zeros((len(images), image_size[0] * image_size[1]))
    iprocessedz = np.zeros((len(images), image_size[0], image_size[1]))
    for i, image in enumerate(images):
        image = image.squeeze()
        image = skimage.filters.gaussian(image, 1)
#         image = skimage.filters.rank.mean(image, skimage.morphology.selem.disk(3))
        image = skimage.transform.resize(image, image_size)
        iprocessedz[i, :, :] = image
        inp_processedz[i, :] = image.flatten()
    return inp_processedz, iprocessedz

def get_hog(images):
    shape = len(skimage.feature.hog(images[0]))
    Xt = np.zeros((len(images), shape))
    for i, image in enumerate(images):
        Xt[i, :] = skimage.feature.hog(image)
    return Xt

def imagets1(images):
    image_size = (40,40)
    X, Ximages = myresize(images, image_size)
    Xt = get_hog(Ximages)
    return Xt

def imagets2(images):
    image_size = (40,40)
    X, Ximages = myresize(images, image_size)
    Xt = get_hog(Ximages)
    Xret = np.concatenate((X, Xt), axis=1)
    return Xret

Xt1 = imagets1(X)

In [5]:
estimators = [ 
                (('reduce_dim'), sklearn.decomposition.PCA()),
                (('clf', sklearn.neural_network.MLPClassifier()))]
pipe = pipeline.Pipeline(estimators)
param_grid = [
    { 
        'reduce_dim__n_components' : [70, 80, 85, 90, 100],
        'clf__hidden_layer_sizes' : [100],
        'clf' : [sklearn.neural_network.MLPClassifier()]  
    },
]

grid_search = sklearn.model_selection.GridSearchCV(pipe, param_grid=param_grid)
grid_search.fit(Xt1, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidde...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'clf__hidden_layer_sizes': [100], 'reduce_dim__n_components': [70, 80, 85, 90, 100], 'clf': [MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=100, learning_rate='constant',
 ...True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
  

In [6]:
print(grid_search.best_score_)
print(grid_search.best_estimator_.steps[0])

0.978947368421
('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False))


In [7]:
pipe = grid_search.best_estimator_
joblib.dump(pipe, 'models/Pipe.p')

['models/Pipe.p']